In [1]:
from llmcoder.analyze import GPTReviewAnalyzer_v1

In [2]:
input = '''        model_first : str, optional
            The model to use for the first completion, by default "ft:gpt-3.5-turbo-1106:personal::8LCi9Q0d"
        model_feedback : str, optional
            The model to use for the feedback loop, by default "gpt-3.5-turbo"
        feedback_variant : str, optional
            The feedback variant to use, by default "separate"
        system_prompt : str, optional
            The system prompt to use, by default the one used for preprocessing and fine-tuning
        max_iter : int, optional
            The maximum number of iterations to run the feedback loop, by default 10
        log_conversation : bool, optional
            Whether to log the conversation, by default False
        """
        if analyzers is None:
            self.analyzers = []
        else:
            self.analyzers = [AnalyzerFactory.create_analyzer(analyzer) for analyzer in analyzers]

        self.model_first = model_first
        self.model_feedback = model_feedback

        self.client = openai.OpenAI(api_key=get_openai_key())
        if feedback_variant not in ["separate", "coworker"]:
            raise ValueError("Inavlid feedback method")

        self.iterations = 0
        self.analyzer_pass_history: list[list[dict]] = []
        self.max_iter = max_iter
        self.feedback_variant = feedback_variant

        if log_conversation:
            self.conversation_file = self._create_conversation_file()
        else:
            self.conversation_file = None  # type: ignore

        self.messages: list = []

        if system_prompt is None:
            self.system_prompt = get_system_prompt()
        elif system_prompt in os.listdir(get_system_prompt_dir()):
            self.system_prompt = get_system_prompt(system_prompt)
        else:
            self.system_prompt = system_prompt

        self._add_message("system", message=self.system_prompt)

    def complete(self, code: str) -> str:
        """
        Complete the provided code with the LLMCoder feedback loop

        Parameters
        ----------
        code : str
            The code to complete

        Returns
        -------
        str
            The completed code
        """
        # Get the first completion with
        self.complete_first(code)

        if len(self.analyzers) > 0:
            # Run the feedback loop until the code is correct or the max_iter is reached
            for _ in range(self.max_iter):
                if self.feedback_step():
                    # If the feedback is correct, break the loop and return the code
                    break

        # Return the last message
        return self.messages[-1]["content"]

    @staticmethod
    def _create_conversation_file() -> str:
        """
        Create the conversation file

        Returns
        -------
        str
            The path of the conversation file
        """
        return os.path.join(get_conversations_dir(create=True), f"{datetime.now()}.jsonl")

    def _add_message(self, role: str, model: str = 'gpt-3.5-turbo', message: str | None = None) -> None:
        """
        Add a message to the messages list

        Parameters
        ----------
        role : str
            The role of the message
        model : str, optional
            The model to use for the completion, by default 'gpt-3.5-turbo'
        message : str, optional
            The message to add, by default None
        """
        # If the user is the assistant, generate a response
        if role == "assistant" and message is None:
            chat_completion = self.client.chat.completions.create(messages=self.messages, model=model)  # type: ignore

            message = chat_completion.choices[0].message.content

        self.messages.append({
            "role": role,
            "content": message,
        })

        if self.conversation_file is not None:
            # If the conversation file already exists, only append the last message as a single line
            if os.path.isfile(self.conversation_file):
                with open(self.conversation_file, "a") as f:
                    f.write(json.dumps(self.messages[-1], ensure_ascii=False) + "\n")
            # Otherwise, write the whole conversation
            else:
                '''

In [3]:
completion = '''open(self.conversation_file, "w") as f:
                    for message in self.messages:
                        f.write(json.dumps(message, ensure_ascii=False) + "\n")'''

In [4]:
# print(analyzer._prompt_template(input, completion))

In [5]:
analyzer = GPTReviewAnalyzer_v1(system_prompt="2023-12-02_GPTReviewAnalyzer_v3.txt")

In [6]:
result = analyzer.analyze('...' + input[-200:], completion)
print(result['pass'])
print()
print(result['message'])

True

The added code in the [COMPLETE] section looks reasonable and addresses the missing part in the [INCOMPLETE] section. However, there are a few improvements that can be made:

1. Syntax Validity: The line `open(self.conversation_file, "w") as f:` is missing the `with` statement. It should be written as `with open(self.conversation_file, "w") as f:` to properly open the file in a context manager.

2. Logical Consistency: It seems that the intention is to write the entire conversation to the file if there is no previous content present. However, the current implementation only writes the last message in the conversation. To fix this, you can modify the loop to write all messages instead of just the last one.

3. Performance Efficiency: Since the file is being opened for writing, it is better to use the mode "w" instead of "a" (append). This will ensure that the file is cleared before writing the new content. If the file contains a large conversation history, this can improve perform

In [7]:
from llmcoder import LLMCoder

In [8]:
llmcoder = LLMCoder(analyzers=["gpt_review_analyzer_v1"], temperature=0.7)

In [9]:
input = '''def _check_hashability(df):
    """This is a static method that checks if a given pandas DataFrame is hashable or not.

    It checks if all the columns containing object types in the input DataFrame are hashable or not.
    If any column is not hashable, it raises a TypeError indicating which columns are not hashable.

    Parameters
    ----------
    df : pandas.DataFrame
        The DataFrame to be checked for hashability.

    Raises
    ------
    TypeError
        If any column containing object types in the input DataFrame is not hashable.
    ""'''

In [10]:
completion = llmcoder.complete(input)

Creating first completion...
Starting feedback loop...
Starting feedback iteration 1...
Analyzing code...
Running GPTReviewAnalyzer_v1...
1 / 1 analyzers passed


In [11]:
for message in llmcoder.messages:
    print(message['role'].upper())
    print(message['content'])
    print('-' * 50)

SYSTEM
You are AutocompleteGPT, a useful AI autocomplete tool that provides code completions based on the user's code.
You are a precision-focused tool for code autocompletion, adept in languages like Python, JavaScript, C++, and SQL.
Precisely continue the code from the point of interruption and do not repeat or modify the original code, even if it is incorrect or the code interrupts in the middle of a line.
Your code is well documented with comments and annotations, and you should provide a clear explanation of the code's purpose in your code completion.
Your unique capability is to provide completions without altering, repeating, or commenting on the original code.
You offer only the necessary code to complete the snippet, ensuring the response is exclusively code, with no additional comments, explanations, or annotations.
This approach makes you an ideal assistant for users seeking straightforward and efficient code extensions, enhancing their work with accurate, logic-driven compl

In [12]:
for analyzer_message in llmcoder.analyzer_message_history:
    print(analyzer_message[0])

The completion is identical to the user's code, so there are no additions or changes to review. The completion is correct and aligned with the user's intent. 

SCORE: 10
